<a href="https://colab.research.google.com/github/vijayamurugan27/GenerativeAI/blob/main/Agents_with_Langraph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Modern AI Pro: Agents with Langraph
This script demonstrates a basic chat agent leveraging the integration of LangChain's LLM capabilities with a search tool.

The agent is designed to handle conversational queries, such as asking about the weather, by utilizing both a memory component for maintaining conversation context and a search tool for retrieving relevant information.

Key Components:
- **LangchainMemory**: Initializes and manages the underlying language model, tailored for the Azure OpenAI deployment. The memory buffer allows the agent to retain context across interactions, improving the quality of responses over time.
- **TavilySearchResults**: A search tool that fetches and returns search results, limited to a configurable maximum number of results. In this demo, it is set to return up to 2 results per query.
- **MemorySaver**: A checkpoint component that saves and restores conversation state, ensuring continuity and consistency across different conversation threads.
- **create_react_agent**: A utility that combines the LLM, search tools, and memory checkpointing into a coherent agent that can process user inputs and return intelligent, context-aware responses.
- **Gradio Interface**: Provides a simple chat interface for interacting with the agent, where users can input messages and receive responses in real-time.

Usage:
- The `chat` function handles streaming responses from the agent. It processes the conversation in chunks to allow for partial responses, yielding the first message content from the agent.
- The `config` object contains a hardcoded thread ID, which is used to manage and switch between different conversation threads.
- The script ends by launching a Gradio interface where users can interact with the agent under the title "Mitra Robot Buffer Memory Chat".


In [2]:
!pip install -U -q langchain_groq langchain_community gradio langgraph
import os

from langchain_groq import ChatGroq
from google.colab import userdata
llm_groq = ChatGroq(model_name="llama3-groq-70b-8192-tool-use-preview", api_key=userdata.get("GROQ_API_KEY"))
os.environ["TAVILY_API_KEY"] = userdata.get("TAVILY_API_SEARCH")

In [3]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent

In [4]:
search = TavilySearchResults(max_results=2)
tools = [search]

# Create the agent
memory = MemorySaver()
agent_executor = create_react_agent(llm_groq, tools, checkpointer=memory)
config = {"configurable": {"thread_id": "id111"}} # Hardcoded -- this is actually to switch different threads of conversations

In [5]:
import gradio as gr
def chat(message, history):
    for chunk in agent_executor.stream(
        {"messages": [HumanMessage(content=message)]}, config):
        agent_data = chunk.get("agent", None)

        if agent_data is not None:
            messages = agent_data.get("messages", None)

        if messages is not None and len(messages) > 0:
            yield messages[0].content

# Setup Gradio
demo = gr.ChatInterface(
    fn=chat,
    title="Mitra Robot Search Engine powered chat",
)
demo.launch()

# You can ask something like:
# What is today's weather in Mumbai?
# Give me the local news there.
# Can you give me today's news in India and say in Navjot Sidhu's tone?

/usr/local/lib/python3.10/dist-packages/gradio/components/chatbot.py:231: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://8cade38f1a3ab0cfb7.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
